# Alberi decisionali per fare previsioni
In questo laboratorio addestriamo un albero decisionale con dati sintetici per un problema di regressione (previsione di un numero reale). In questo esempio, simuliamo la previsione della dimensione media dei pacchetti di un flusso data la durata di quel flusso.

In [ ]:
# Author: Roberto Doriguzzi-Corin
# Project: Corso di Algoritmi di Machine Learning per la rilevazione di attacchi informatici
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#   http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Import necessary libraries
import numpy as np
from sklearn.tree import DecisionTreeRegressor, plot_tree
import matplotlib.pyplot as plt

OUTPUT_FILE = "./reg_tree"

# Generate some sample data (a sin function with some random noise)
np.random.seed(0)
m = 100 
X_train = 12 * np.random.rand(m, 1) +8
y_train = (-(X_train-8)**4 + 50000 + 3000*np.random.randn(m, 1))/10000-2

np.random.seed(1)
X_test = 12 * np.random.rand(m, 1) +8
y_test = (-(X_test-8)**4 + 50000 + 3000*np.random.randn(m, 1))/10000-2

# Mostriamo i campioni che abbiamo raccolto
Questi campioni rappresentano il packet rate (pacchetti al secondo) rilevati nella rete a diverse ore del giorni. Questi campioni posso essere usati per addestrare un algoritmo di machine learning. In questo esempio, addestreremo un albero decisionale.

In [ ]:
# Plot the results
plt.figure()
plt.scatter(X_train, y_train, s=20, edgecolor="black", c="yellow", label="data")
plt.xlabel("Ora del giorno")
plt.ylabel("Packet Rate")
plt.title("Volume di traffico di rete")
plt.legend()
plt.show()

# Albero decisionale per prevedere il traffico di rete ad ogni ora del giorno
Usiamo questi dati per prevedere il volume di traffico che ci aspettiamo a qualsiasi ora del giorno compresa tra le 8 di mattina e le 8 di sera. Il modello mi dice il volume di traffico che mi aspetto normalmente, se non ci sono attacchi di rete.

In [ ]:
# Fit a decision tree regressor
tree_regr = DecisionTreeRegressor(max_depth=2) # try with min_samples_leaf to avoid overfitting
tree_regr.fit(X_train, y_train)

# Plot the decision tree
plt.figure(figsize=(10, 6))
plot_tree(tree_regr, filled=True, feature_names=["Ora del giorno"])
plt.title("Albero decisionale")
plt.show()

# Visualizziamo come l'albero decisionale effettua le previsioni
Provate a cambiare i parametri dell'albero decisionale (es: ```max_depth```, ```min_samples_split``` e ```min_samples_leaf```) e cercate di capire come cambia il modello.
Se aumentate la dimensione dell'albero (numero di palchi), il modello migliora sempre?

In [ ]:
# Predictions (numbers from 0 to 20 with increment 0.01)
X_plot = np.arange(8, 20, 0.01)[:, np.newaxis]
y_pred = tree_regr.predict(X_plot)

# Plot the results
plt.figure()
plt.scatter(X_train, y_train, s=20, edgecolor="black", c="yellow", label="Campioni di addestramento")
plt.plot(X_plot, y_pred, color="cornflowerblue", label="modello")
plt.xlabel("Ora del giorno")
plt.ylabel("Packet Rate")
plt.title("Volume di traffico di rete")
plt.legend()
plt.show()

# Misuriamo come il modello rappresenta i campioni di addestramento
Per fare questo, usiamo i campioni di addestramento per fare delle previsioni con l'albero decisionale. Confrontriamo poi il risultato con il valore atteso e facciamo la differenza. Questa differenza (errore) ci dice quanto "bene" ha imparato il modello. Piu' piccolo e' l'errore, meglio e'.


In [ ]:
from sklearn.metrics import mean_squared_error

y_pred = tree_regr.predict(X_train)
mse = mean_squared_error(y_train,y_pred)
print("Errore nelle previsioni sui campioni usati per l'addestramento: ", mse)

# Vediamo come il modello si comporta con dei campioni di traffico normale mai visti
In questo caso usiamo degli altri campioni che rappresentazione delle rilevazioni del packet rate in condizioni normali (nessun attacco). Questi campioni non sono stati usati per l'addestramento e ci servono per capire se il modello ha imparato bene.

In [ ]:
X_plot = np.arange(8, 20, 0.01)[:, np.newaxis]
y_pred = tree_regr.predict(X_plot)

plt.figure()
plt.scatter(X_test, y_test, s=20, edgecolor="black", c="yellow", label="Campioni di test")
plt.plot(X_plot, y_pred, color="cornflowerblue", label="modello")
plt.xlabel("Ora del giorno")
plt.ylabel("Packet Rate")
plt.title("Volume di traffico di rete")
plt.legend()
plt.show()

y_pred = tree_regr.predict(X_test)
mse = mean_squared_error(y_test,y_pred)
print("Errore nelle previsioni sui campioni di test: ", mse)

# Rilevazione delle anomalie con l'albero decisionale
Ora usiamo l'albero decisionale che abbiamo addestrato per rilevare le anomalie di rete. In questo caso vogliamo rilevare un packet rate piu' alto rispetto al normale. Fenomeno che potrebbe indicare un attacco di rete di tipo DDoS volumetrico. Prima generiamo e visualizziamo le anomalie.

In [ ]:
# Generiamo alcune anomalie
a = 10
X_train_anomaly = 2 * np.random.rand(a, 1) +16
y_train_anomaly = (-(X_train_anomaly-8)**4 + 50000 + 3000*np.random.randn(a, 1))/10000-2 + 1

X_plot = np.arange(8, 20, 0.01)[:, np.newaxis]
y_pred = tree_regr.predict(X_plot)

plt.figure()
plt.scatter(X_train, y_train, s=20, edgecolor="black", c="yellow", label="Dati normali")
plt.scatter(X_train_anomaly, y_train_anomaly, s=20, edgecolor="black", c="red", label="Anomalie")
plt.plot(X_plot, y_pred, color="cornflowerblue", label="modello")
plt.xlabel("Ora del giorno")
plt.ylabel("Packet Rate")
plt.title("Volume di traffico di rete")
plt.legend()
plt.show()

# Misura dell'errore
Misurariamo l'errore di previsione sui campioni anomali. E' piu' alto o piu' basso?

In [ ]:
y_pred = tree_regr.predict(X_train_anomaly)
mse = mean_squared_error(y_train_anomaly,y_pred)
print("Errore sui campioni anomali: ", mse)